In [1]:
from gadopt import *
from gadopt.inverse import *

/home/riik/gadopt/firedrake/lib/python3.10/site-packages/pytools/persistent_dict.py:52: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "
firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [2]:
# define the helmholtz solver
def helmholtz(V, source):
    u = Function(V)
    v = TestFunction(V)
    F = inner(grad(v), grad(u)) * dx + 100.0*v*u*dx - v*source*dx

    solve(F == 0, u)
    return u

## Define the forward problem and the objective functional

In [3]:
# define a mesh
mesh = UnitIntervalMesh(10)
num_processes = mesh.comm.size
mesh_checkpoint = f"mesh_helmholtz_np{num_processes}.h5"
# create a checkpointable mesh by writing to disk and restoring
with CheckpointFile(mesh_checkpoint, "w") as f:
    f.save_mesh(mesh)
with CheckpointFile(mesh_checkpoint, "r") as f:
    mesh = f.load_mesh("firedrake_default")

# define the space and sources
V = FunctionSpace(mesh, "CG", 1)
source_ref = Function(V)
x = SpatialCoordinate(mesh)
source_ref.interpolate(cos(pi * x**2))

# compute reference solution
with stop_annotating():
    u_ref = helmholtz(V, source_ref)

# tape the forward solution
source = Function(V)
c = Control(source)
u = helmholtz(V, source)

# define the reduced objective functional
J = assemble(1e6 * (u - u_ref)**2 * dx)
rf = ReducedFunctional(J, c)

# define the boundary conditions
T_lb = Function(V, name="Lower bound")
T_ub = Function(V, name="Upper bound")
T_lb.assign(-1.0)
T_ub.assign(1.0)

Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x71fb6f1ccf40>, FiniteElement('Lagrange', interval, 1), name=None), Mesh(VectorElement(FiniteElement('Lagrange', interval, 1), dim=1), 3)), 36)

## Using Lin-more optimiser

In [4]:
# define the optimiser run
def run(optimiser, rf, rank, filename):
    if rank == 0:
        with open(filename, "w") as f:
            rf.eval_cb_post = lambda val, *args: f.write(f"{val}\n")
            optimiser.run()
            rf.eval_cb_pots = lambda *args: None
    else:
        optimiser.run()

# set up the minimisation problem
minimisation_problem = MinimizationProblem(rf, bounds=(T_lb, T_ub))
minimisation_parameters["Status Test"]["Iteration Limit"] = 10

# run full optimisation, checkpointing every iteration
checkpoint_dir = f"optimisation_checkpoint_np{num_processes}"
optimiser = LinMoreOptimiser(
    minimisation_problem,
    minimisation_parameters,
    checkpoint_dir=checkpoint_dir,
)
run(optimiser, rf, mesh.comm.rank, f"full_optimisation_np{num_processes}.dat")


Lin-More Trust-Region Method (Type B, Bound Constraints)
  iter  value          gnorm          snorm          delta          #fval     #grad     #hess     #proj     tr_flag   iterCG    flagCG    
  0     5.093612e+01   9.660918e-01   ---            1.000000e+00   1         1         0         2         ---       ---       ---       
  1     4.449608e+00   1.861899e+00   9.660918e-01   1.000000e+00   2         2         5         9         0         0         0         
  2     4.217224e-01   1.199692e+00   2.307581e-01   1.000000e+01   3         3         12        15        0         5         0         
  3     1.463350e-01   8.921325e-01   6.159620e-02   1.000000e+02   4         4         20        20        0         5         0         
  4     7.202067e-02   6.472893e-01   3.783784e-02   1.000000e+03   5         5         29        25        0         6         0         
  5     4.475407e-02   8.469459e-01   3.115330e-02   1.000000e+04   6         6         39        30        

## Using Scipy minimize

In [7]:
# Setting up the problem using minimize that uses Scipy
sol = minimize(rf, bounds=(T_lb, T_ub), tol=1e-12)

ValueError: I/O operation on closed file.